步骤 1 - 在 Google Colab 上运行 Ollama
开始打开一个新的 Google Colab 笔记本。为了确保我们正在使用 GPU，点击菜单中的“运行时”，选择“更改运行时类型”，并选择“T4 GPU”作为硬件加速器。
现在，我们需要安装与 Ngrok 相关的依赖项，并配置 authtoken。

In [ ]:
!pip install aiohttp pyngrok                
!ngrok config add-authtoken [2jGwvYUFzOlOqXi4LtZhEz5br7W_6FL4uUfUqsT942i4xTvE7]

使用以下命令安装 Ollama

In [ ]:
!curl https://ollama.ai/install.sh |sh

现在，是时候在 NGrok 旁边运行 Ollama 了。在这个例子中，我们将配置 Ollama 同时运行 Mistral 和 Codellama 模型。这个设置允许我们以后在不同模型之间切换，为我们的聊天应用程序的功能提供了灵活性。
为了管理这些，我们将它们作为单独的子进程运行：
•Ollama 将启动模型。localhost:11434。
•NGrok 然后会暴露端口11434连接到互联网使用公共 URL。
执行以下代码片段以启动 Ollama 和 NGrok：

In [ ]:
import os                
import asyncio                

# Set LD_LIBRARY_PATH so the system NVIDIA library                
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

In [ ]:
asyncdef run_process(cmd):                  
  print('>>> starting', *cmd)                  
  p = await asyncio.subprocess.create_subprocess_exec(     
      *cmd,
      stdout=asyncio.subprocess.PIPE,                      
      stderr=asyncio.subprocess.PIPE,                  
  )                

asyncdef pipe(lines):                    
      asyncfor line in lines:                      
      print(line.strip().decode('utf-8'))                


await asyncio.gather(                      
      pipe(p.stdout),                      
      pipe(p.stderr),                  
)                


await asyncio.gather(                    
    run_process(['ollama', 'serve']),                    
    run_process(['ollama', 'pull', 'gemma2']),                    
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),    
)